In [2]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly

In [3]:
engine = create_engine("sqlite:///mta_data.db")

In [4]:
engine.table_names()

['mta_data']

In [5]:
df = pd.read_sql('SELECT * FROM mta_data ORDER BY RANDOM() LIMIT 100000;', engine)
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,R333,R366,00-00-00,225 ST,25,IRT,04/02/2021,05:00:00,REGULAR,7769062,2390743
1,S102,R165,00-03-01,TOMPKINSVILLE,1,SRT,06/02/2021,18:00:00,REGULAR,58223,2
2,R109,R305,03-00-02,WTC-CORTLANDT,1,IRT,03/27/2021,01:00:00,REGULAR,396675,165490
3,R206,R014,02-00-02,FULTON ST,2345ACJZ,IRT,09/29/2021,05:00:00,REGULAR,1631389,4252262
4,H022,R279,00-00-01,JEFFERSON ST,L,BMT,03/27/2021,12:00:00,REGULAR,2994087,1011708


In [6]:
df = df.rename(columns={'C/A': 'control_area', 'UNIT': 'unit', 'SCP': 'subunit_channel_pos', 'STATION':'station', 'LINENAME':'subway_lines', 'DIVISION':'division', 'DATE':'date', 'TIME':'time', 'DESC':'desc', 'ENTRIES':'entries', 'EXITS':'exits'})

In [7]:
df= df.sort_values(by=['control_area', 'unit', 'subunit_channel_pos', 'station', 'date', 'time'])

In [8]:
df['date_time'] = df.date + ' ' + df.time

In [9]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time
6017,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2021,15:00:00,REGULAR,7532471,2568387,02/17/2021 15:00:00
65403,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2021,12:00:00,REGULAR,7552051,2577847,04/01/2021 12:00:00
11529,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/22/2021,00:00:00,REGULAR,7560222,2582362,04/22/2021 00:00:00
61034,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2021,00:00:00,REGULAR,7561288,2582898,04/24/2021 00:00:00
54689,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/01/2021,12:00:00,REGULAR,7564691,2584460,05/01/2021 12:00:00


In [10]:
df['net_entries'] = df.entries.diff()
df['net_exits'] = df.exits.diff()
df['net_traffic'] = df.entries.diff() + df.exits.diff()

In [11]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic
6017,A002,R051,02-00-00,59 ST,NQR456W,BMT,02/17/2021,15:00:00,REGULAR,7532471,2568387,02/17/2021 15:00:00,NaN,NaN,NaN
65403,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/01/2021,12:00:00,REGULAR,7552051,2577847,04/01/2021 12:00:00,19580.0,9460.0,29040.0
11529,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/22/2021,00:00:00,REGULAR,7560222,2582362,04/22/2021 00:00:00,8171.0,4515.0,12686.0
61034,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/24/2021,00:00:00,REGULAR,7561288,2582898,04/24/2021 00:00:00,1066.0,536.0,1602.0
54689,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/01/2021,12:00:00,REGULAR,7564691,2584460,05/01/2021 12:00:00,3403.0,1562.0,4965.0


In [12]:
df.date = pd.to_datetime(df['date'])

In [13]:
df.time = pd.to_datetime(df['time'])

In [14]:
df.date_time = pd.to_datetime(df['date_time'])

In [15]:
df['audit_length_hours'] = df.date_time.diff() / pd.Timedelta(hours=1)

In [16]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic,audit_length_hours
6017,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-02-17,2022-01-04 15:00:00,REGULAR,7532471,2568387,2021-02-17 15:00:00,NaN,NaN,NaN,NaN
65403,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-01,2022-01-04 12:00:00,REGULAR,7552051,2577847,2021-04-01 12:00:00,19580.0,9460.0,29040.0,1029.0
11529,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-22,2022-01-04 00:00:00,REGULAR,7560222,2582362,2021-04-22 00:00:00,8171.0,4515.0,12686.0,492.0
61034,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-24,2022-01-04 00:00:00,REGULAR,7561288,2582898,2021-04-24 00:00:00,1066.0,536.0,1602.0,48.0
54689,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-05-01,2022-01-04 12:00:00,REGULAR,7564691,2584460,2021-05-01 12:00:00,3403.0,1562.0,4965.0,180.0


In [17]:
df['avg_hourly_entries'] = df['net_entries'] / df['audit_length_hours']
df['avg_hourly_exits'] = df['net_exits'] / df['audit_length_hours']
df['avg_hourly_traffic'] = df['net_traffic'] / df['audit_length_hours']

In [18]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,exits,date_time,net_entries,net_exits,net_traffic,audit_length_hours,avg_hourly_entries,avg_hourly_exits,avg_hourly_traffic
6017,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-02-17,2022-01-04 15:00:00,REGULAR,7532471,2568387,2021-02-17 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65403,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-01,2022-01-04 12:00:00,REGULAR,7552051,2577847,2021-04-01 12:00:00,19580.0,9460.0,29040.0,1029.0,19.028183,9.193392,28.221574
11529,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-22,2022-01-04 00:00:00,REGULAR,7560222,2582362,2021-04-22 00:00:00,8171.0,4515.0,12686.0,492.0,16.607724,9.176829,25.784553
61034,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-24,2022-01-04 00:00:00,REGULAR,7561288,2582898,2021-04-24 00:00:00,1066.0,536.0,1602.0,48.0,22.208333,11.166667,33.375000
54689,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-05-01,2022-01-04 12:00:00,REGULAR,7564691,2584460,2021-05-01 12:00:00,3403.0,1562.0,4965.0,180.0,18.905556,8.677778,27.583333


In [38]:
df['pct_change_entries'] = df['net_entries'].pct_change()
df['pct_change_exits'] = df['net_exits'].pct_change()
df['pct_change_traffic'] = df['net_traffic'].pct_change()

In [39]:
df.head()

,control_area,unit,subunit_channel_pos,station,subway_lines,division,date,time,desc,entries,...,net_entries,net_exits,net_traffic,audit_length_hours,avg_hourly_entries,avg_hourly_exits,avg_hourly_traffic,pct_change_entries,pct_change_exits,pct_change_traffic
6017,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-02-17,2022-01-04 15:00:00,REGULAR,7532471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65403,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-01,2022-01-04 12:00:00,REGULAR,7552051,...,19580.0,9460.0,29040.0,1029.0,19.028183,9.193392,28.221574,NaN,NaN,NaN
11529,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-22,2022-01-04 00:00:00,REGULAR,7560222,...,8171.0,4515.0,12686.0,492.0,16.607724,9.176829,25.784553,-0.582686,-0.522727,-0.563154
61034,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-04-24,2022-01-04 00:00:00,REGULAR,7561288,...,1066.0,536.0,1602.0,48.0,22.208333,11.166667,33.375000,-0.869539,-0.881285,-0.873719
54689,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-05-01,2022-01-04 12:00:00,REGULAR,7564691,...,3403.0,1562.0,4965.0,180.0,18.905556,8.677778,27.583333,2.192308,1.914179,2.099251


In [41]:
df['turnsitle'] = df.subunit_channel_pos

In [46]:
df = df.drop(labels=['control_area', 'unit', 'subunit_channel_pos', 'division', 'date', 'time', 'desc', 'entries', 'exits'],axis=1)

In [47]:
cols = df.columns.tolist()

In [48]:
cols

['station',
 'subway_lines',
 'date_time',
 'net_entries',
 'net_exits',
 'net_traffic',
 'audit_length_hours',
 'avg_hourly_entries',
 'avg_hourly_exits',
 'avg_hourly_traffic',
 'pct_change_entries',
 'pct_change_exits',
 'pct_change_traffic',
 'turnsitle']

In [50]:
cols = ['turnstile', 'station',
 'subway_lines',
 'date_time',
 'net_entries',
 'net_exits',
 'net_traffic',
 'audit_length_hours',
 'avg_hourly_entries',
 'avg_hourly_exits',
 'avg_hourly_traffic',
 'pct_change_entries',
 'pct_change_exits',
 'pct_change_traffic']

In [53]:
cols

['turnstile',
 'station',
 'subway_lines',
 'date_time',
 'net_entries',
 'net_exits',
 'net_traffic',
 'audit_length_hours',
 'avg_hourly_entries',
 'avg_hourly_exits',
 'avg_hourly_traffic',
 'pct_change_entries',
 'pct_change_exits',
 'pct_change_traffic']

In [56]:
df.head()

,station,subway_lines,date_time,net_entries,net_exits,net_traffic,audit_length_hours,avg_hourly_entries,avg_hourly_exits,avg_hourly_traffic,pct_change_entries,pct_change_exits,pct_change_traffic,turnsitle
6017,59 ST,NQR456W,2021-02-17 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02-00-00
65403,59 ST,NQR456W,2021-04-01 12:00:00,19580.0,9460.0,29040.0,1029.0,19.028183,9.193392,28.221574,NaN,NaN,NaN,02-00-00
11529,59 ST,NQR456W,2021-04-22 00:00:00,8171.0,4515.0,12686.0,492.0,16.607724,9.176829,25.784553,-0.582686,-0.522727,-0.563154,02-00-00
61034,59 ST,NQR456W,2021-04-24 00:00:00,1066.0,536.0,1602.0,48.0,22.208333,11.166667,33.375000,-0.869539,-0.881285,-0.873719,02-00-00
54689,59 ST,NQR456W,2021-05-01 12:00:00,3403.0,1562.0,4965.0,180.0,18.905556,8.677778,27.583333,2.192308,1.914179,2.099251,02-00-00
